In [8]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import requests
import math
import pandas as pd
from IPython.display import display, HTML
from IPython.display import display_html

class ConsensusFromWeb(object):
    def __init__(self, day):
        self._checkConsensusStatus = 0
        self._isTreasuryClicked = 1
        self._isEbitdaClicked = 2

        self._chrome_path = "C:/Users/jihod/Quant/Kiwoom/Data/chromedriver.exe"
        self._driver = webdriver.Chrome(self._chrome_path)
        self._delay = 10
        self._day = day


    def parse_data(self,stock,menuNum, tabNum,optionNum,isNeedPreProcess=False):
        self._preparationByTab(stock, menuNum,tabNum,optionNum,isNeedPreProcess)
        return self.get_data(stock, menuNum)

    def get_data(self, stock, menuNum):
        try:
            html_source = self._driver.page_source     
            if menuNum==1:
                text_split_first = html_source.split('익계산서,재무상태표,현금흐름표에')[2]
            if menuNum==2:
                text_split_first = html_source.split('데이터를 보여줍니다.')[2]
            

            text_split_second = text_split_first.split("table")[0]

            finalstring = "<table"+text_split_second+"table>"
            finalstring = finalstring.replace("display: none", "display:block")
            crypto_data = pd.read_html(finalstring)
            crypto_data = crypto_data[0]
            crypto_data = crypto_data.rename(
                columns={'항목': '재무'})
            df = crypto_data.set_index('재무')
            colNameList = df.columns.tolist()
            if "2018/03" in colNameList[0]:
                df = df.rename(
                    columns={colNameList[0]: "2018-03-31", colNameList[1]: "2018-06-30",colNameList[2]: "2018-09-30", 
                             colNameList[3]: "2018-12-31", colNameList[4]:'2019-03-31',colNameList[5]:'2019-06-30'})
            elif "2018/06" in colNameList[0]:
                df = df.rename(
                    columns={colNameList[0]: "2018-06-30",colNameList[1]: "2018-09-30", colNameList[2]: "2018-12-31"
                             , colNameList[3]:'2019-03-31',colNameList[4]:'2019-06-30'})
            elif "2018/09" in colNameList[0]:
                df = df.rename(
                    columns={colNameList[0]: "2018-09-30", colNameList[1]: "2018-12-31"
                             , colNameList[2]:'2019-03-31',colNameList[3]:'2019-06-30', colNameList[4]:'2019-09-30', colNameList[5]:'2019-12-31'})        
            elif "2018/12" in colNameList[0]:
                df = df.rename(
                    columns={colNameList[0]: "2018-12-31"
                             , colNameList[1]:'2019-03-31',colNameList[2]:'2019-06-30',colNameList[3]:'2019-09-30',colNameList[4]:'2019-12-31'}) 
            elif "2019/03" in colNameList[0]:
                df = df.rename(
                    columns={colNameList[0]: "2019-03-31"
                             , colNameList[1]:'2019-06-30',colNameList[2]:'2019-09-30',colNameList[3]:'2019-12-31',colNameList[4]:'2020-3-31'})                
            self._dropUnneededCol(df)
                
            return df
        except TimeoutException:
            print ("로딩이 제대로 되어지지 않았습니다.")
        return -1

        
    def _preparationByTab(self, stock, menuNum,tabNum, optionNum, isNeedPreProcess):
        ##selenium을 쓰기 위한 초기설정.
        if(menuNum !=1 and menuNum !=2):
            raise Exception("menu 값이 잘못되었습니다 (1~2)")
        if(tabNum !=1 and tabNum !=2 and tabNum !=3):
            raise Exception("tab 값이 잘못되었습니다 (1~3)")
        if(optionNum !=1 and optionNum !=2 and optionNum !=3 and optionNum !=4 and optionNum !=5):
            raise Exception("option 값이 잘못되었습니다 (1~5)")
        if menuNum==1: 
            newUrl = 'https://comp.wisereport.co.kr/company/c1030001.aspx?cmp_cd='+stock[1:]+'&cn='      
            self._driver.get(newUrl)
            if isNeedPreProcess:
                if tabNum ==1:
                    self._driver.find_element_by_xpath('//*[@id="rpt_tab1"]').click()
                if tabNum ==2:
                    self._driver.find_element_by_xpath('//*[@id="rpt_tab2"]').click()
                if tabNum ==3:
                    self._driver.find_element_by_xpath('//*[@id="rpt_tab3"]').click()
        if menuNum==2: 
            newUrl = 'https://comp.wisereport.co.kr/company/c1040001.aspx?cmp_cd='+stock[1:]+'&cn='      
            self._driver.get(newUrl)
            if isNeedPreProcess:
                if tabNum ==1:
                    self._driver.find_element_by_xpath('//*[@id="val_tab1"]').click()
                if tabNum ==2:
                    self._driver.find_element_by_xpath('//*[@id="val_tab2"]').click()
                if tabNum ==3:
                    self._driver.find_element_by_xpath('//*[@id="val_tab3"]').click()
        if isNeedPreProcess:
            self._driver.find_element_by_xpath('//*[@id="finGubun"]/option[' + str(optionNum)+']').click()   
            self._driver.find_element_by_xpath('//*[@id="wrapper"]/table[2]/tbody/tr/td[2]/dl/dt[3]/label').click()
            self._driver.find_element_by_xpath('//*[@id="hfinGubun"]').click()
        myElem = WebDriverWait(self._driver, self._delay).until(EC.presence_of_element_located((By.CLASS_NAME, 'cut')))

       
            
    def _tryGettingData(self, df, 재무지표, 단위=100000000):
        ##데이터 가져오기(없으면 0 처리)
        값 = 0
        try:
            값 = df.at[재무지표]
            if isinstance(값, str):
                if(값=="완전잠식"):
                    return "완전잠식"
            값 = float(값) *단위
            
        except:
            return 0

        
        if math.isnan(값):
            값 = 0
        return 값
    
    def _dropUnneededCol(self, df):
        ##필요없는 컬럼 제거
        colList = df.columns.tolist()
        if 6 in colList:
            df = df.drop(6, axis=1)
        if 7 in colList:
            df = df.drop(7, axis=1)
        if 8 in colList:
            df = df.drop(8, axis=1)

In [9]:
import sqlite3
import requests
class ConsensusToExcelWriter(object):
    def __init__(self, load_wb, write_wb, day):
        ##error list에 대해서 다시 파싱해주는 미구현 기능
        self._errorList = dict()
        self._load_wb = load_wb
        self._write_wb = write_wb
        self._dest_filename = "C:/Users/jihod/Quant/Kiwoom/Data/dataoverride4.xlsx"
        self._load_ws = self._load_wb['Sheet1']
        self._write_ws = self._write_wb['Sheet1']
        self._sheet = "Sheet1"
        self._day = day      
        #크롤링 시작점. 재무지표에 따라 바뀜
        self._codeCellItr = 2

        #현재 크롤링중인 지표이름
        self._consensus = ""
        
        #한 재무지표가 끝나면 selenium 세팅 변경
        self._needToChange = False
        
        ##최대 데이타 요청 횟수
        self._defaultTryCount = 5
        
        ##crawler
        self._crawler = ConsensusFromWeb(day)
    def _setConsensus(self, 재무지표):
        self._consensus = 재무지표
        self._needToChange = True
        
        if 재무지표 == "지배주주순이익":
            self._codeCellItr = 2
        elif 재무지표 == "지배주주지분":
            self._codeCellItr = 3
        elif 재무지표 == "자기주식":
            self._codeCellItr = 4
        elif 재무지표 == "현금흐름":
            self._codeCellItr = 5
        elif 재무지표 == "매출액":
            self._codeCellItr = 6
        elif 재무지표 == "총부채":
            self._codeCellItr = 7
        elif 재무지표 == "EBITDA":
            self._codeCellItr = 8
            
    def _findWholeStock(self):
        con = sqlite3.connect("C:/Users/jihod/Quant/Kiwoom/Data/상장종목검색.db")

        cursor = con.cursor()
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        list_index = cursor.fetchall()
        newList  = []
        for item in list_index:
            newList.append(item[0])
        return newList
       
    def _findStockInFile(self):
        self._sheet = "Sheet1"
        oldCode = "A005930"
        newCode = ""
        codeCount = 0
        list = []
        self._codeCellItr = 2
        while(True):
            oldCode = newCode
            newCode = self._load_ws.cell(9,self._codeCellItr).value
            if(oldCode != newCode):
                list.append(newCode)

            self._codeCellItr+=1
            if self._checkSheetChange():
                break
        return list
    
    def _findNewlyAddedStock(self):
        
        url = 'https://finance.naver.com/sise/sise_new_stock.nhn?sosok=0'
        response = requests.get(url)
        text = response.text
        text_split_first = text.split('/item/main.nhn?code=')
        newlyAddedStock = []
        for i in range(99):
            code = "A" + text_split_first[i+1][0:6]
            newlyAddedStock.append(code)
        url = 'https://finance.naver.com/sise/sise_new_stock.nhn?sosok=1'
        response = requests.get(url)
        text = response.text
        text_split_first = text.split('/item/main.nhn?code=')
        for i in range(99):
            code = "A" + text_split_first[i+1][0:6]
            newlyAddedStock.append(code)

        return newlyAddedStock
    
    def _checkSheetChange(self):
        if(self._load_ws.cell(9,self._codeCellItr).value==None):
            self._setConsensus(self._consensus)     
            if(self._sheet=="Sheet2"):
                self._load_ws = self._load_wb['Sheet1']
                self._write_ws = self._write_wb['Sheet1']
                print(self._consensus + '완료')
                self._sheet ="Sheet1"
                return True
            self._load_ws = self._load_wb['Sheet2']
            self._write_ws = self._write_wb['Sheet2']
            self._sheet ="Sheet2"
            return False
        
    def writeNewlyAddedStock(self):
        self._sheet = "Sheet1"
        self._load_ws = self._load_wb["Sheet1"]
        newStockList = self._findNewlyAddedStock()
        fileStockList = self._findStockInFile()
        wholeStockList = self._findWholeStock()
        self._codeCellItr = 2
        unaddedNewStockList = list(set(newStockList)-set(fileStockList))
        
        newStockListFinal = list(set(unaddedNewStockList).intersection(wholeStockList))
        newStockListFinal = sort(newStockListFinal)

        codeCount = 0
        while(True):
            findBlank = self._load_ws.cell(row = 9, column = self._codeCellItr)
            if findBlank.value==None:
                break
            self._codeCellItr+=1
        for item in newStockListFinal:
            print("새로 상장된 종목 추가 : ", item)
            writeNewCode =  self._load_ws.cell(row = 9, column = self._codeCellItr)
            writeNewCode.value = item
            writeNewCode =  self._load_ws.cell(row = 13, column = self._codeCellItr)
            writeNewCode.value = "지배주주순이익(원)"

            writeNewCode =  self._load_ws.cell(row = 9, column = self._codeCellItr+1)
            writeNewCode.value = item
            writeNewCode =  self._load_ws.cell(row = 13, column = self._codeCellItr+1)
            writeNewCode.value = "지배주주지분(원)"
            
            writeNewCode =  self._load_ws.cell(row = 9, column = self._codeCellItr+2)
            writeNewCode.value = item
            writeNewCode =  self._load_ws.cell(row = 13, column = self._codeCellItr+2)
            writeNewCode.value = "자기주식(원)"

            writeNewCode =  self._load_ws.cell(row = 9, column = self._codeCellItr+3)
            writeNewCode.value = item
            writeNewCode =  self._load_ws.cell(row = 13, column = self._codeCellItr+3)
            writeNewCode.value = "현금흐름(원)"
            
            writeNewCode =  self._load_ws.cell(row = 9, column = self._codeCellItr+4)
            writeNewCode.value = item
            writeNewCode =  self._load_ws.cell(row = 13, column = self._codeCellItr+4)
            writeNewCode.value = "매출액(원)"
            
            writeNewCode =  self._load_ws.cell(row = 9, column = self._codeCellItr+5)
            writeNewCode.value = item
            writeNewCode =  self._load_ws.cell(row = 13, column = self._codeCellItr+5)
            writeNewCode.value = "총부채(원)"
            
            writeNewCode =  self._load_ws.cell(row = 9, column = self._codeCellItr+6)
            writeNewCode.value = item
            writeNewCode =  self._load_ws.cell(row = 13, column = self._codeCellItr+6)
            writeNewCode.value = "EBITDA(원)"
            self._codeCellItr+=7

        self._load_wb.save(filename = "C:/Users/jihod/Quant/Kiwoom/Data/재무데이터(분기).xlsx")
        self._codeCellItr = 2
                
    def write(self, 재무지표, isDoubleCheck = False):
        self._setConsensus(재무지표)
        self._sheet = "Sheet1"
        oldCode = "A005930"
        newCode = ""
        codeCount = 0
        while(True):
            oldCode = newCode

            newCode = self._load_ws.cell(9,self._codeCellItr).value
            
            if isDoubleCheck:
                if newCode not in list(self._errorList.keys()):
                    self._codeCellItr +=7
                    if self._checkSheetChange():
                        break
                else:
                    if 재무지표 not in self._errorList[newCode] :
                        self._codeCellItr +=7
                        if self._checkSheetChange():
                            break
            print(newCode," : ",codeCount," sheet :", self._sheet ," cellWriteLoc row 92 and col: ", self._codeCellItr)
            tryCount = self._defaultTryCount
            if(oldCode != newCode):
                ##접속 문제인 경우 10번까지 시도, 안되면 에러가 있는 종목(상장폐지 등)
                while(True):
                    if(tryCount<0):
                        tryCount = self._defaultTryCount
                        writeValue = None
                        self._needToChange = True
                        break
                    try:
                        writeValue = self._findWriteCSS(재무지표, newCode)
                        self._needToChange = False
                        break
                    except:
                        writeValue = 0
                        print("warning :: Try again-- ", newCode)
                        tryCount-=1
                codeCount+=1

            cellForWrite = self._write_ws.cell(row = 94, column = self._codeCellItr)
            
            if writeValue!=None:
                cellForWrite.value = writeValue

            self._codeCellItr+=7
            self._write_wb.save(filename = self._dest_filename)
            if self._checkSheetChange():
                break
            
        
    def _findWriteCSS(self, 재무지표, newCode):
        if (재무지표 == '(지배주주지분)당기순이익') or (재무지표 =="지배주주순이익") or (재무지표 =='지배주주순이익(원)'):
            df = self._crawler.parse_data(newCode, 1,1,1, self._needToChange)[self._day]
            return self._crawler._tryGettingData(df, '(지배주주지분)당기순이익')

        elif 재무지표 == '지배주주지분(원)' or 재무지표 =='지배주주지분' or 재무지표 =='자본총계(지배)＜당기>':
            df = self._crawler.parse_data(newCode, 2,1,1, self._needToChange)[self._day]
            return self._crawler._tryGettingData(df, '자본총계(지배)＜당기＞')

        elif 재무지표 == '현금흐름' or 재무지표 == '현금흐름(원)':
            df = self._crawler.parse_data(newCode, 1,3,1, self._needToChange)[self._day] 

            ## 현금흐름 = 당기순이익 - 비지배주주순이익 + 감가상각비 + 무형자산상각비
            
            retValue = self._write_ws.cell(row = 94, column = self._codeCellItr-3).value
            if self._crawler._tryGettingData(df, '감가상각비'):
                return retValue + self._crawler._tryGettingData(df, '감가상각비') + \
                        self._crawler._tryGettingData(df, '무형자산상각비') + \
                        self._crawler._tryGettingData(df, '기타무형자산상각비')
            else:
                return retValue + \
                        self._crawler._tryGettingData(df, '유형자산등감가상각비') + \
                        self._crawler._tryGettingData(df, '유형자산감가상각비') + \
                        self._crawler._tryGettingData(df, '리스(사용권)자산감가상각비') +  \
                        self._crawler._tryGettingData(df, '렌탈자산감가상각비') + \
                        self._crawler._tryGettingData(df, '무형자산상각비') + \
                        self._crawler._tryGettingData(df, '기타무형자산상각비')
        elif 재무지표 == '매출액' or 재무지표 == '매출액(원)' or 재무지표 =='매출액(수익)':
            df = self._crawler.parse_data(newCode, 1,1,1, self._needToChange)[self._day] 
            if "은행" in newCode or "금융" in newCode or "지주" in newCode:
                return self._crawler._tryGettingData(df, '이자수익')
            if "화재" in newCode or "보험" in newCode or "생명" in newCode:
                return self._crawler._tryGettingData(df, '보험료수익')
            if "카드" in newCode or "증권" in newCode or "캐피탈" in newCode:
                return self._crawler._tryGettingData(df, '순영업이익')        
            if "신탁" in newCode :
                return self._crawler._tryGettingData(df, '영업수익')         
            return self._crawler._tryGettingData(df, '펼치기  매출액(수익)')
        elif 재무지표 == '총부채' or 재무지표 == '총부채(원)' or 재무지표 =='부채총계': 
            df = self._crawler.parse_data(newCode, 1,2,1, self._needToChange)[self._day] 
            return self._crawler._tryGettingData(df, '부채총계')

        elif 재무지표 == 'EBITDA':
            df = self._crawler.parse_data(newCode, 1,1,1, self._needToChange)[self._day] 



            return self._crawler._tryGettingData(df, '영업이익') + \
                    self._write_ws.cell(row = 94, column = self._codeCellItr-3).value - \
                    self._write_ws.cell(row = 94, column = self._codeCellItr-6).value
        
        elif 재무지표 == '자기주식': 
            df = self._crawler.parse_data(newCode, 1,2,1, self._needToChange)[self._day] 
            return self._crawler._tryGettingData(df, '자기주식')
        
    def aversionCheck(self, 재무지표, isDoubleCheck=False, errorBound = 0.15):
        self._setConsensus(재무지표)
        oldCode = "A005930"
        self._sheet="Sheet1"
        newCode = ""
        codeCount = 0
        correctDataCount = 0
        while(True):
            oldCode = newCode

            newCode = self._load_ws.cell(9,self._codeCellItr).value

            tryCount = self._defaultTryCount
#             if(oldCode != newCode):

            cellWritten = self._write_ws.cell(row = 92, column = self._codeCellItr)
            cellOrigin = self._load_ws.cell(row = 89, column = self._codeCellItr)
            checkCond = cellWritten.value != None and \
                        cellOrigin.value != None and \
                        cellOrigin.value !='' and \
                        cellWritten.value !='' and \
                        cellOrigin.value !='완전잠식' and \
                        cellWritten.value !='완전잠식'

            if checkCond:
                cellWritten.value = int(cellWritten.value)
                cellOrigin.value = int(cellOrigin.value)
           
                print("cellOrigin.value : " , str(cellOrigin.value))
                print("cellWritten.value : " , str(cellWritten.value))

                if abs(cellWritten.value) != 0:
                    codeCount+=1
                    if cellOrigin.value  == 0:
                        if newCode not in list(self._errorList.keys()):
                            self._errorList[newCode] = [재무지표]
                        else:
                            if 재무지표 not in self._errorList[newCode]:
                                self._errorList[newCode].append(재무지표)
                    elif abs(cellWritten.value - cellOrigin.value)/abs(cellWritten.value) > errorBound:
                        if newCode not in list(self._errorList.keys()):
                            self._errorList[newCode] = [재무지표]
                        else:
                            if 재무지표 not in self._errorList[newCode]:
                                self._errorList[newCode].append(재무지표)
                    elif abs(cellWritten.value - cellOrigin.value)/abs(cellWritten.value) <= errorBound:
                        correctDataCount+=1
                        if isDoubleCheck:
                            if newCode in list(self._errorList.keys()):
                                if 재무지표 in self._errorList[newCode]:
                                    self._errorList[newCode].remove(재무지표)
                                    if len(self._errorList[newCode])==0:
                                        del self._errorList[newCode]
            self._codeCellItr+=7

            if self._checkSheetChange():
                break
        print(재무지표+" 데이타 정합율 : ", str(correctDataCount/ codeCount))

In [10]:
from openpyxl import load_workbook

start = time.time()
_override_path = "C:/Users/jihod/Quant/Kiwoom/Data/dataoverride4.xlsx"
#data_only=Ture로 해줘야 수식이 아닌 값으로 받아온다.
load_wb = load_workbook("C:/Users/jihod/Quant/Kiwoom/Data/재무데이터(분기).xlsx", data_only=True)
write_wb = load_workbook(_override_path, data_only=True)

res = len(write_wb.sheetnames)
if res==1:
    write_wb.create_sheet('Sheet2')
    write_wb.save(_override_path)

In [ ]:
writer = ConsensusToExcelWriter(load_wb, write_wb, "2019-12-31")

# writer.writeNewlyAddedStock()

writer.write("지배주주순이익")


writer.write("지배주주지분")

writer.write("현금흐름")
writer.write("매출액")

writer.write("총부채")

writer.write("EBITDA")

writer.write("자기주식")


A005930  :  0  sheet : Sheet1  cellWriteLoc row 92 and col:  2
A000660  :  1  sheet : Sheet1  cellWriteLoc row 92 and col:  9
A005380  :  2  sheet : Sheet1  cellWriteLoc row 92 and col:  16
A068270  :  3  sheet : Sheet1  cellWriteLoc row 92 and col:  23
A051910  :  4  sheet : Sheet1  cellWriteLoc row 92 and col:  30
A055550  :  5  sheet : Sheet1  cellWriteLoc row 92 and col:  37
A012330  :  6  sheet : Sheet1  cellWriteLoc row 92 and col:  44
A017670  :  7  sheet : Sheet1  cellWriteLoc row 92 and col:  51
A005490  :  8  sheet : Sheet1  cellWriteLoc row 92 and col:  58
A051900  :  9  sheet : Sheet1  cellWriteLoc row 92 and col:  65
A207940  :  10  sheet : Sheet1  cellWriteLoc row 92 and col:  72
A105560  :  11  sheet : Sheet1  cellWriteLoc row 92 and col:  79
A035420  :  12  sheet : Sheet1  cellWriteLoc row 92 and col:  86
A028260  :  13  sheet : Sheet1  cellWriteLoc row 92 and col:  93
A034730  :  14  sheet : Sheet1  cellWriteLoc row 92 and col:  100
A000270  :  15  sheet : Sheet1  cell

A004370  :  121  sheet : Sheet1  cellWriteLoc row 92 and col:  849
A010120  :  122  sheet : Sheet1  cellWriteLoc row 92 and col:  856
A181710  :  123  sheet : Sheet1  cellWriteLoc row 92 and col:  863
A003240  :  124  sheet : Sheet1  cellWriteLoc row 92 and col:  870
A009420  :  125  sheet : Sheet1  cellWriteLoc row 92 and col:  877
A161890  :  126  sheet : Sheet1  cellWriteLoc row 92 and col:  884
A051600  :  127  sheet : Sheet1  cellWriteLoc row 92 and col:  891
A139130  :  128  sheet : Sheet1  cellWriteLoc row 92 and col:  898
A006280  :  129  sheet : Sheet1  cellWriteLoc row 92 and col:  905
A204320  :  130  sheet : Sheet1  cellWriteLoc row 92 and col:  912
A214320  :  131  sheet : Sheet1  cellWriteLoc row 92 and col:  919
A042670  :  132  sheet : Sheet1  cellWriteLoc row 92 and col:  926
A005300  :  133  sheet : Sheet1  cellWriteLoc row 92 and col:  933
A020560  :  134  sheet : Sheet1  cellWriteLoc row 92 and col:  940
A000670  :  135  sheet : Sheet1  cellWriteLoc row 92 and col: 

A003230  :  242  sheet : Sheet1  cellWriteLoc row 92 and col:  1696
A005690  :  243  sheet : Sheet1  cellWriteLoc row 92 and col:  1703
A001720  :  244  sheet : Sheet1  cellWriteLoc row 92 and col:  1710
A271980  :  245  sheet : Sheet1  cellWriteLoc row 92 and col:  1717
A003850  :  246  sheet : Sheet1  cellWriteLoc row 92 and col:  1724
A004430  :  247  sheet : Sheet1  cellWriteLoc row 92 and col:  1731
A002990  :  248  sheet : Sheet1  cellWriteLoc row 92 and col:  1738
A019680  :  249  sheet : Sheet1  cellWriteLoc row 92 and col:  1745
A053210  :  250  sheet : Sheet1  cellWriteLoc row 92 and col:  1752
A010050  :  251  sheet : Sheet1  cellWriteLoc row 92 and col:  1759
A108670  :  252  sheet : Sheet1  cellWriteLoc row 92 and col:  1766
A025860  :  253  sheet : Sheet1  cellWriteLoc row 92 and col:  1773
A298050  :  254  sheet : Sheet1  cellWriteLoc row 92 and col:  1780
A007570  :  255  sheet : Sheet1  cellWriteLoc row 92 and col:  1787
A000370  :  256  sheet : Sheet1  cellWriteLoc ro

A000020  :  360  sheet : Sheet1  cellWriteLoc row 92 and col:  2522
A026890  :  361  sheet : Sheet1  cellWriteLoc row 92 and col:  2529
A161000  :  362  sheet : Sheet1  cellWriteLoc row 92 and col:  2536
A095570  :  363  sheet : Sheet1  cellWriteLoc row 92 and col:  2543
A001940  :  364  sheet : Sheet1  cellWriteLoc row 92 and col:  2550
A001200  :  365  sheet : Sheet1  cellWriteLoc row 92 and col:  2557
A004710  :  366  sheet : Sheet1  cellWriteLoc row 92 and col:  2564
A267850  :  367  sheet : Sheet1  cellWriteLoc row 92 and col:  2571
A000680  :  368  sheet : Sheet1  cellWriteLoc row 92 and col:  2578
A079980  :  369  sheet : Sheet1  cellWriteLoc row 92 and col:  2585
A036580  :  370  sheet : Sheet1  cellWriteLoc row 92 and col:  2592
A004700  :  371  sheet : Sheet1  cellWriteLoc row 92 and col:  2599
A015360  :  372  sheet : Sheet1  cellWriteLoc row 92 and col:  2606
A272550  :  373  sheet : Sheet1  cellWriteLoc row 92 and col:  2613
A092230  :  374  sheet : Sheet1  cellWriteLoc ro

A013570  :  478  sheet : Sheet1  cellWriteLoc row 92 and col:  3348
A001460  :  479  sheet : Sheet1  cellWriteLoc row 92 and col:  3355
A004200  :  480  sheet : Sheet1  cellWriteLoc row 92 and col:  3362
A002760  :  481  sheet : Sheet1  cellWriteLoc row 92 and col:  3369
A017900  :  482  sheet : Sheet1  cellWriteLoc row 92 and col:  3376
A005950  :  483  sheet : Sheet1  cellWriteLoc row 92 and col:  3383
A008970  :  484  sheet : Sheet1  cellWriteLoc row 92 and col:  3390
A000390  :  485  sheet : Sheet1  cellWriteLoc row 92 and col:  3397
A000230  :  486  sheet : Sheet1  cellWriteLoc row 92 and col:  3404
A003460  :  487  sheet : Sheet1  cellWriteLoc row 92 and col:  3411
A024090  :  488  sheet : Sheet1  cellWriteLoc row 92 and col:  3418
A000890  :  489  sheet : Sheet1  cellWriteLoc row 92 and col:  3425
A267290  :  490  sheet : Sheet1  cellWriteLoc row 92 and col:  3432
A014790  :  491  sheet : Sheet1  cellWriteLoc row 92 and col:  3439
A000490  :  492  sheet : Sheet1  cellWriteLoc ro

...

A078150  :  1343  sheet : Sheet2  cellWriteLoc row 92 and col:  1374
A110790  :  1344  sheet : Sheet2  cellWriteLoc row 92 and col:  1381
A104830  :  1345  sheet : Sheet2  cellWriteLoc row 92 and col:  1388
A023160  :  1346  sheet : Sheet2  cellWriteLoc row 92 and col:  1395
A052400  :  1347  sheet : Sheet2  cellWriteLoc row 92 and col:  1402
A084370  :  1348  sheet : Sheet2  cellWriteLoc row 92 and col:  1409
A067390  :  1349  sheet : Sheet2  cellWriteLoc row 92 and col:  1416
A051370  :  1350  sheet : Sheet2  cellWriteLoc row 92 and col:  1423
A200670  :  1351  sheet : Sheet2  cellWriteLoc row 92 and col:  1430
A255440  :  1352  sheet : Sheet2  cellWriteLoc row 92 and col:  1437
A140860  :  1353  sheet : Sheet2  cellWriteLoc row 92 and col:  1444
A950170  :  1354  sheet : Sheet2  cellWriteLoc row 92 and col:  1451
A039440  :  1355  sheet : Sheet2  cellWriteLoc row 92 and col:  1458
A086390  :  1356  sheet : Sheet2  cellWriteLoc row 92 and col:  1465
A054950  :  1357  sheet : Sheet2  

A005990  :  1459  sheet : Sheet2  cellWriteLoc row 92 and col:  2186
A033160  :  1460  sheet : Sheet2  cellWriteLoc row 92 and col:  2193
A073560  :  1461  sheet : Sheet2  cellWriteLoc row 92 and col:  2200
A115180  :  1462  sheet : Sheet2  cellWriteLoc row 92 and col:  2207
A196490  :  1463  sheet : Sheet2  cellWriteLoc row 92 and col:  2214
A018310  :  1464  sheet : Sheet2  cellWriteLoc row 92 and col:  2221
A131390  :  1465  sheet : Sheet2  cellWriteLoc row 92 and col:  2228
A108790  :  1466  sheet : Sheet2  cellWriteLoc row 92 and col:  2235
A137940  :  1467  sheet : Sheet2  cellWriteLoc row 92 and col:  2242
A006620  :  1468  sheet : Sheet2  cellWriteLoc row 92 and col:  2249
A054050  :  1469  sheet : Sheet2  cellWriteLoc row 92 and col:  2256
A016100  :  1470  sheet : Sheet2  cellWriteLoc row 92 and col:  2263
A166480  :  1471  sheet : Sheet2  cellWriteLoc row 92 and col:  2270
A033110  :  1472  sheet : Sheet2  cellWriteLoc row 92 and col:  2277
A023600  :  1473  sheet : Sheet2  

A035080  :  1578  sheet : Sheet2  cellWriteLoc row 92 and col:  3019
A004650  :  1579  sheet : Sheet2  cellWriteLoc row 92 and col:  3026
A071460  :  1580  sheet : Sheet2  cellWriteLoc row 92 and col:  3033
A222800  :  1581  sheet : Sheet2  cellWriteLoc row 92 and col:  3040
A161580  :  1582  sheet : Sheet2  cellWriteLoc row 92 and col:  3047
A014470  :  1583  sheet : Sheet2  cellWriteLoc row 92 and col:  3054
A900300  :  1584  sheet : Sheet2  cellWriteLoc row 92 and col:  3061
A226340  :  1585  sheet : Sheet2  cellWriteLoc row 92 and col:  3068
A061040  :  1586  sheet : Sheet2  cellWriteLoc row 92 and col:  3075
A032790  :  1587  sheet : Sheet2  cellWriteLoc row 92 and col:  3082
A056700  :  1588  sheet : Sheet2  cellWriteLoc row 92 and col:  3089
A072870  :  1589  sheet : Sheet2  cellWriteLoc row 92 and col:  3096
A079370  :  1590  sheet : Sheet2  cellWriteLoc row 92 and col:  3103
A060230  :  1591  sheet : Sheet2  cellWriteLoc row 92 and col:  3110
A005710  :  1592  sheet : Sheet2  

A051160  :  1694  sheet : Sheet2  cellWriteLoc row 92 and col:  3831
A049520  :  1695  sheet : Sheet2  cellWriteLoc row 92 and col:  3838
A215790  :  1696  sheet : Sheet2  cellWriteLoc row 92 and col:  3845
A067310  :  1697  sheet : Sheet2  cellWriteLoc row 92 and col:  3852
A187220  :  1698  sheet : Sheet2  cellWriteLoc row 92 and col:  3859
A047770  :  1699  sheet : Sheet2  cellWriteLoc row 92 and col:  3866
A149950  :  1700  sheet : Sheet2  cellWriteLoc row 92 and col:  3873
A198440  :  1701  sheet : Sheet2  cellWriteLoc row 92 and col:  3880
A093190  :  1702  sheet : Sheet2  cellWriteLoc row 92 and col:  3887
A065170  :  1703  sheet : Sheet2  cellWriteLoc row 92 and col:  3894
A013990  :  1704  sheet : Sheet2  cellWriteLoc row 92 and col:  3901
A073110  :  1705  sheet : Sheet2  cellWriteLoc row 92 and col:  3908
A073570  :  1706  sheet : Sheet2  cellWriteLoc row 92 and col:  3915
A053350  :  1707  sheet : Sheet2  cellWriteLoc row 92 and col:  3922
A042510  :  1708  sheet : Sheet2  

A900270  :  1813  sheet : Sheet2  cellWriteLoc row 92 and col:  4664
A208640  :  1814  sheet : Sheet2  cellWriteLoc row 92 and col:  4671
A204630  :  1815  sheet : Sheet2  cellWriteLoc row 92 and col:  4678
A131290  :  1816  sheet : Sheet2  cellWriteLoc row 92 and col:  4685
A043340  :  1817  sheet : Sheet2  cellWriteLoc row 92 and col:  4692
A041460  :  1818  sheet : Sheet2  cellWriteLoc row 92 and col:  4699
A197210  :  1819  sheet : Sheet2  cellWriteLoc row 92 and col:  4706
A290550  :  1820  sheet : Sheet2  cellWriteLoc row 92 and col:  4713
A094940  :  1821  sheet : Sheet2  cellWriteLoc row 92 and col:  4720
A092460  :  1822  sheet : Sheet2  cellWriteLoc row 92 and col:  4727
A026040  :  1823  sheet : Sheet2  cellWriteLoc row 92 and col:  4734
A104460  :  1824  sheet : Sheet2  cellWriteLoc row 92 and col:  4741
A079960  :  1825  sheet : Sheet2  cellWriteLoc row 92 and col:  4748


In [6]:
writer = ConsensusToExcelWriter(load_wb, write_wb, "2020-03-31")


In [7]:
writer._crawler.parse_data("A005930", 1,2,1,True)[writer._day]["부채총계"]

KeyError: '2020-03-31'

In [15]:
writer = ConsensusToExcelWriter(load_wb, write_wb, "2019-12-31")


In [18]:
writer._crawler.parse_data("A005930", 1,2,1,True)[writer._day]["부채총계"]

896840.8

In [17]:
writer._crawler.parse_data("A005930", 1,2,1,True)

,2018-12-31,2019-03-31,2019-06-30,2019-09-30,2019-12-31,전분기대비(QoQ),전년동기대비(YoY),전분기대비(QoQ).1,전년동기대비(YoY).1,Unnamed: 10
재무,,,,,,,,,,
자산총계,3393572.4,3450678.8,3429401.3,3533859.9,3525645.0,NaN,-0.2,3.9,NaN,NaN
유동자산,1746974.2,1773885.2,1734334.8,1860421.3,1813852.6,NaN,-2.5,3.8,NaN,NaN
펼치기 재고자산,289847.0,314559.9,312469.9,309087.6,267664.6,NaN,-13.4,-7.7,NaN,NaN
상품,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
제품,88361.0,82908.1,85579.0,NaN,81151.2,NaN,NaN,-8.2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
수권주식수,25000000.0,25000000.0,25000000.0,25000000.0,25000000.0,NaN,0.0,0.0,NaN,NaN
1주당금액,100.0,100.0,100.0,100.0,100.0,NaN,0.0,0.0,NaN,NaN
발행주식수,6792669.0,6792669.0,6792669.0,6792669.0,6792669.0,NaN,0.0,0.0,NaN,NaN


In [17]:
writer._crawler.parse_data("A005930", 1,1,1)

,2018-12-31,2019-03-31,2019-06-30,2019-09-30,2019/12(IFRS연결) 분기컨센서스보기,전분기대비(QoQ),전년동기대비(YoY),전분기대비(QoQ).1,전년동기대비(YoY).1,Unnamed: 10
재무,,,,,,,,,,
펼치기 매출액(수익),592650.5,523855.5,561271.0,620034.7,598847.6,566226.9,-3.4,1.1,-5.5,8.1
*내수,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
*수출,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
제품매출액,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
상품매출액,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
*희석주당순이익,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
*(지배주주지분)주당계속사업이익,NaN,NaN,746.0,899.0,NaN,NaN,NaN,NaN,NaN,NaN
*(지배주주지분)주당순이익,NaN,NaN,746.0,899.0,NaN,781.0,NaN,NaN,NaN,NaN
